In [1]:
def box_contains_centroid(box, centroid):
    return centroid[0] >= box[0] and centroid[0] <= box[2] and centroid[1] >= box[1] and centroid[1] <= box[3]

def get_centroid_of_box(box):
    return box[0]+(box[2]-box[0])/2, box[1]+(box[3]-box[1])/2

In [2]:
import os

from PIL import Image
import PIL.ImageDraw as ImageDraw
import numpy as np
import time

from keras_yolo3.yolo import YOLO

annotation_path = './data/train.txt'
with open(annotation_path) as f:
    lines = f.readlines()
      
detector = YOLO()

#for line in lines:
line = lines[0]
line = line.split()
image = Image.open(line[0])
iw, ih = image.size
h, w = (416, 416)
gt_boxes = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])

draw = ImageDraw.Draw(image)

/home/sean/.virtualenvs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


/home/sean/Documents/indystudy/data/trained_weights_final.h5 model, anchors, and classes loaded.


In [3]:
predicted_boxes = []
for y in range(0, ih, h):
    for x in range(0, iw, w):
        area = (x, y, x+w, y+h)
        window = image.crop(area)

        # predicted bounding boxes
        _, window_predictions = detector.detect_image_silent(window)

        for bb in window_predictions:
            adjusted_bb = [bb['left'] + x, bb['top'] + y, bb['right'] + x, bb['bottom'] + y]
            
            predicted_boxes.append(adjusted_bb)
            
            draw.rectangle(adjusted_bb, outline="blue")

(416, 416, 3)
Found 0 boxes for img
2.432921545998397
(416, 416, 3)
Found 2 boxes for img
0.9056873059998907
(416, 416, 3)
Found 1 boxes for img
0.9179386449995945
(416, 416, 3)
Found 2 boxes for img
0.9836239349988318
(416, 416, 3)
Found 4 boxes for img
0.9675745800013829
(416, 416, 3)
Found 4 boxes for img
0.9566851419986051
(416, 416, 3)
Found 1 boxes for img
0.978573670001424
(416, 416, 3)
Found 1 boxes for img
0.9614044270001614
(416, 416, 3)
Found 1 boxes for img
1.0244627179999952
(416, 416, 3)
Found 0 boxes for img
0.9684257899989461
(416, 416, 3)
Found 1 boxes for img
1.0670496740003728
(416, 416, 3)
Found 1 boxes for img
1.130755088001024
(416, 416, 3)
Found 3 boxes for img
0.9449369720005052
(416, 416, 3)
Found 3 boxes for img
0.921446886000922
(416, 416, 3)
Found 3 boxes for img
0.9798969529983879
(416, 416, 3)
Found 1 boxes for img
0.9829092840009253
(416, 416, 3)
Found 1 boxes for img
0.9347393480002211
(416, 416, 3)
Found 1 boxes for img
1.0811771389999194
(416, 416, 3)


Found 20 boxes for img
0.9282864199994947
(416, 416, 3)
Found 20 boxes for img
0.931595264999487
(416, 416, 3)
Found 6 boxes for img
0.9177873830012686
(416, 416, 3)
Found 0 boxes for img
0.9213119609994465
(416, 416, 3)
Found 1 boxes for img
0.9118577600002027
(416, 416, 3)
Found 7 boxes for img
0.936325743999987
(416, 416, 3)
Found 17 boxes for img
0.9330097029996978
(416, 416, 3)
Found 10 boxes for img
0.9260785229998874
(416, 416, 3)
Found 16 boxes for img
0.9320471150003868
(416, 416, 3)
Found 20 boxes for img
0.9311067039998306
(416, 416, 3)
Found 14 boxes for img
0.9272141920009744
(416, 416, 3)
Found 15 boxes for img
0.8968078630005039
(416, 416, 3)
Found 13 boxes for img
0.9163711339988367
(416, 416, 3)
Found 15 boxes for img
0.9247550910004065
(416, 416, 3)
Found 13 boxes for img
0.9191383759989549
(416, 416, 3)
Found 20 boxes for img
0.9085321590009698
(416, 416, 3)
Found 17 boxes for img
0.9344544610012235
(416, 416, 3)
Found 20 boxes for img
0.9288443939985882
(416, 416, 3

In [4]:
# true bounding boxes
false_negatives = 0
false_positives = 0
true_positives = 0
chopped_detections = 0

for bb in gt_boxes:
    
    preds_for_gt = []
    for pred in predicted_boxes:
        if box_contains_centroid(bb, get_centroid_of_box(pred)):
            preds_for_gt.append(pred)
            # remove predition to increase efficiency

    if len(preds_for_gt) == 0: # nothing detected for gt
        false_negatives += 1

        draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="red")
    elif len(preds_for_gt) > 1: # multiple boxes for one gt
        chopped_detections += 1

        draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="yellow")
    else:
        true_positives += 1

        draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="green")

        # calculate IoU
            
# once finished, remaining preditions will be false_positives

print(" False Negatives:", false_negatives, "\n",
      "Chopped Detections:", chopped_detections, "\n",
      "True Positives:", true_positives)
    
image.save('first_train_example_detections_with_gt.tiff')
#break


 False Negatives: 100 
 Chopped Detections: 390 
 True Positives: 1958
